# Знакомство с word2vec


## Установка необходимых библиотек

Для Gensim нужна специфичная версия Numpy, который в свою очередь много где используется, поэтому после установки перезапустим среду, чтобы использовать именно нужную версию. На ошибки версий других библиотек внимание можно не обращать

In [ ]:
import warnings
warnings.filterwarnings('ignore')
!pip install -q numpy==1.24.4
import os
os.kill(os.getpid(), 9)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 29.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
blosc2 3.3.4 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
xarray-einstats 0.9.0 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
pymc 5.23.0 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.


In [ ]:
!pip install -q scipy==1.10.1
!pip install -q gensim==4.3.3
import gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 42.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
xarray-einstats 0.9.0 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
xarray-einstats 0.9.0 requires scipy>=1.11, but you have scipy 1.10.1 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
cvxpy 1.6.5 requires scipy>=1.11.0, but you have scipy 1.10.1 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.1

## Загрузка модели
Скачаем модель <code>google-news-vectors</code>. Откроем ее с помощью библиотеки <code>gensim</code>.

In [75]:
! gdown --id 0B7XkCwpI5KDYNlNUTTlSS21pQmM
! gunzip GoogleNews-vectors-negative300.bin.gz

from gensim.models import KeyedVectors

w = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin",
                                      binary=True)

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
From (redirected): https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM&confirm=t&uuid=35f64e15-e6fc-4375-9cf7-f508dd538076
To: /content/GoogleNews-vectors-negative300.bin.gz
100% 1.65G/1.65G [00:26<00:00, 62.3MB/s]
gzip: GoogleNews-vectors-negative300.bin already exists; do you wish to overwrite (y or n)? y


Если с Google скачать модель не удается, можно загрузить ее и из Kaggle (расскоментируйте и запустите код). Для этого потребуется ввсти в процссе скачивания логин и токен (можно найти в ЛК на Kaggle).

In [ ]:
# ! pip install -q opendatasets
# import opendatasets as od


# od.download(
#     "https://www.kaggle.com/datasets/leadbest/googlenewsvectorsnegative300")

# # Следует ввест логин и токен с Kaggle

# w = KeyedVectors.load_word2vec_format("/content/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin",
#                                       binary=True)


Структура называется <code>KeyedVectors</code> и по сути представляет собой отображение между ключами и векторами. Каждый вектор идентифицируется своим ключом поиска, чаще всего коротким строковым токеном, поэтому обычно это соответствие между

<center><code>{str => 1D numpy array}</code></center><br/>



Например, выведем первые 10 координат вектора, соответствующего слову <code>sunrise</code>

In [ ]:
print("Размерность вектора: ", w["sunrise"].shape)
print("Первые 10 координат вектора: \n", w["sunrise"][:10])

Размерность вектора:  (300,)
Первые 10 координат вектора: 
 [-0.22558594 -0.03540039 -0.21679688  0.03613281 -0.2265625  -0.09814453
  0.109375   -0.34570312  0.18652344  0.01806641]


## Задание 1. Сходство.

Извлеките векторы слов <code>London</code>, <code>England</code>, <code>Moscow</code>. Посчитайте косинусное расстояние между словами <code>London</code> и <code>England</code> и между словами <code>Moscow</code> и <code>England</code>. Какая пара слов ближе? Подсказка: для вычисления косинусного расстояния использвется метод <code>distance()</code>. Правильный ответ представлен в блоке вывода.

In [ ]:
#enter your code here
print("London = > England:", w.distance("London", "England"))
print("Moscow = > England:", w.distance("Moscow", "England"))

London = > England: 0.5600714385509491
Moscow = > England: 0.8476868271827698


## Задание 2. Аналогии.
С помощью метода most_similar решите аналогию
```London : England = Moscow : X```

Правильный ответ представлен в блоке вывода.

(Подсказка: нужно использовать аргументы positive и negative)

In [13]:
#enter your code here
w.most_similar(positive=["Moscow", "England"], negative=["London"])

[('Russia', 0.6502718329429626),
 ('Ukraine', 0.5879061818122864),
 ('Belarus', 0.5666376352310181),
 ('Azerbaijan', 0.5418694615364075),
 ('Armenia', 0.5300518870353699),
 ('Poland', 0.5253247618675232),
 ('coach_Georgy_Yartsev', 0.5220180749893188),
 ('Russian', 0.5214669108390808),
 ('Croatia', 0.5166040658950806),
 ('Moldova', 0.5125792026519775)]

## Задание 3. Сходство: найти лишнее.
С помощью метода <code>doesnt_match</code> найдите лишнее слово в ряду <code>breakfast cereal dinner lunch</code>.

Правильный ответ представлен в блоке вывода.

In [15]:
#enter your code here
w.doesnt_match("breakfast cereal dinner lunch".split())

'cereal'

## Задание 4. Представление предложений в виде векторов


Дано предложение: <code>the quick brown fox jumps over the lazy dog</code>. Вам нужно представить это предложение в виде вектора. Для этого найдите вектор каждого слова в модели, а затем усредните векторы покомпонентно.


In [79]:
import numpy as np

#enter your code here
sentance = "the quick brown fox jumps over the lazy dog"
# vector = None
# for word in sentance.split():
#   if vector is None:
#     vector = w[word].copy()
#   else:
#     vector += w[word]
# (vector / len(sentance.split()))[0:5]
vector = []
for word in sentance.split():
    vector.append(w[word].copy())
vector = np.array(vector).mean(0)
vector


array([ 9.05558243e-02,  5.43416329e-02, -6.71386719e-02,  1.09686956e-01,
       -1.06065534e-02, -1.21066622e-01,  4.63748500e-02, -5.35685234e-02,
        7.00683594e-02,  9.72764790e-02,  2.70589199e-02, -1.16495766e-01,
        3.48307304e-02, -2.13351771e-02, -8.32519531e-02, -2.97851562e-02,
       -3.11482754e-02,  1.02077909e-01, -7.70467147e-02, -1.05170354e-01,
       -8.54492188e-04,  6.69555664e-02,  1.97482631e-02,  7.00336043e-03,
        1.32700605e-02,  2.12593079e-02, -1.07652456e-01,  1.05970591e-01,
        1.06550425e-01,  4.36740462e-03, -5.31684011e-02,  6.63248673e-02,
        3.62277552e-02, -6.70505092e-02, -2.00195312e-02, -1.75272617e-02,
       -2.21082903e-02,  6.37478312e-04,  9.91753489e-02,  1.46647140e-01,
        9.37500000e-02, -1.67263448e-01,  1.36345759e-01, -1.23155378e-02,
        5.61930351e-02, -6.90680593e-02, -5.95092773e-03, -5.62099889e-02,
        8.73616561e-02,  7.48155415e-02, -3.18332240e-02,  7.09228516e-02,
        7.61583149e-02,  

# Сравнение двух моделей

## Загрузка ещё одной модели


Откроем модель google-news-vectors и модель, обученную на британском национальном корпусе http://vectors.nlpl.eu/repository/20/0.zip, с помощью gensim.


In [27]:
! wget -c http://vectors.nlpl.eu/repository/20/0.zip
! unzip 0.zip
! head -3 model.txt

--2025-06-05 10:42:28--  http://vectors.nlpl.eu/repository/20/0.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.200, 2001:700:112::200
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.200|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344050746 (328M) [application/zip]
Saving to: ‘0.zip’

0.zip               100%[===================>] 328.11M  18.6MB/s    in 19s     

2025-06-05 10:42:48 (17.3 MB/s) - ‘0.zip’ saved [344050746/344050746]

Archive:  0.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  
163473 300
say_VERB -0.008861 0.097097 0.100236 0.070044 -0.079279 0.000923 -0.012829 0.064301 -0.029405 -0.009858 -0.017753 0.063115 0.033623 0.019805 0.052704 -0.100458 0.089387 -0.040792 -0.088936 0.110212 -0.044749 0.077675 -0.017062 -0.063745 -0.009502 -0.079371 0.066952 -0.070209 0.063761 -0.038194 -0.046252 0.049983 -0.094985 -0.

Загрузим модель, обученную на британском национальном корпусе

In [28]:
w_british = KeyedVectors.load_word2vec_format("model.bin", binary=True)

Заметим, что размерность векторов в этом случае также равна 300. При этом через нижнее подчеркивание нужно указывать часть речи используемого слова. Слова следует приводить к нижнему регистру.

In [29]:
try:
    print(w_british["London_NOUN"].shape)
    print('upper is ok')
except:
    print(w_british["london_NOUN"].shape)
    print('lower is ok')

(300,)
lower is ok


## Набор данных для оценки качества
Скачаем датасет wordsim353.



In [38]:
# ! wget -c http://alfonseca.org/pubs/ws353simrel.tar.gz
! tar -xvf ws353simrel.tar.gz
! head -5 wordsim353_sim_rel/wordsim_similarity_goldstandard.txt

wordsim353_sim_rel/wordsim353_agreed.txt
wordsim353_sim_rel/wordsim353_annotator1.txt
wordsim353_sim_rel/wordsim353_annotator2.txt
wordsim353_sim_rel/wordsim_relatedness_goldstandard.txt
wordsim353_sim_rel/wordsim_similarity_goldstandard.txt
tiger	cat	7.35
tiger	tiger	10.00
plane	car	5.77
train	car	6.31
television	radio	6.77


## Подготовка эталонной выборки


Из файла `wordsim_similarity_goldstandard.txt` извлечем пары слов и посчитаем косинусное сходство их векторов в обеих моделях. Посчитаем корреляцию оценок сходства в модели google-news-vectors с оценками аннотаторов в датасете, а затем - корреляцию сходства в модели на основе британского национального корпуса с оценками аннотаторов в датасете. Какая модель ближе к суждениям экспертов-разметчиков?

(используем только те слова из wordsim, для которых находятся векторы на британском корпусе, помеченные как существительные!)

In [39]:
import pandas as pd

df = pd.read_csv("wordsim353_sim_rel/wordsim_similarity_goldstandard.txt",
                 sep="\t", header=None)
df.columns = ["first", "second", "score"]
df.head(3)

,first,second,score
0,tiger,cat,7.35
1,tiger,tiger,10.00
2,plane,car,5.77


## Вычисление оценок similarity моделей
Используем только те слова из wordsim, для которых находятся векторы на британском корпусе, помеченные как существительные, сформируйте 3 массива с оценкам схожести:

1. Оценки (косинус между векторами), полученные в результате модели google-news-vectors

2. Оценки (косинус между векторами) полученные в результате модели на основе британского национального корпуса

3. Эталонные оценки из word_sim, для слов из которых находятся векторы на британском корпусе

Пропущенные слова из word_sim представлены в блоке вывода.

In [48]:
gn_dist, br_dist, scores = [], [], []

for row in df.iterrows():

  w1, w2, s = row[1]["first"], row[1]["second"], row[1]["score"]
  try:
    #enter your code here
    word_1 = f"{w1.lower()}_NOUN"
    word_2 = f"{w2.lower()}_NOUN"
    br_dist.append(w_british.distance(word_1, word_2))
    gn_dist.append(w.distance(w1, w2))
    scores.append(s)

  except KeyError as e:
    print(e, "Skipping this word.")

"Key 'stupid_NOUN' not present" Skipping this word.
"Key 'arafat_NOUN' not present" Skipping this word.
"Key 'harvard_NOUN' not present" Skipping this word.
"Key 'mexico_NOUN' not present" Skipping this word.
"Key 'live_NOUN' not present" Skipping this word.
"Key 'seven_NOUN' not present" Skipping this word.
"Key 'five_NOUN' not present" Skipping this word.
"Key 'mars_NOUN' not present" Skipping this word.


## Выбор модели: корреляция с экспертами

Вычислите корреляцию Спирмена для каждой модели по сравнению с эталонными оценками из word_sim.

Результаты представлены в блоке вывода.

In [56]:
from scipy.stats import spearmanr

#enter your code here
print("GN spearmanr corr:", abs(spearmanr(gn_dist, scores).statistic))
print("British spearmanr corr:", abs(spearmanr(br_dist, scores).statistic))

GN spearmanr corr: 0.7817164245392594
British spearmanr corr: 0.762755193448961


Можно заметить, что модель google-news-vectors несколько выигрывает в данном случае.

### 1. Определите косинусное расстояние между векторами слов: tiger и plane

In [67]:
WORD_1 ="tiger"
WORD_2 = "plane"
print("GN %.3f" % w.distance(WORD_1, WORD_2))
print("BR: %.3f" % w_british.distance(f"{WORD_1.lower()}_NOUN", f"{WORD_2.lower()}_NOUN"))

GN 0.834
BR: 0.841


### 2. Дан набор слов: tiger plane train television, определите лишнее слово.

In [97]:
WORDS = "tiger plane train television".split()
words_dist = {}

for w1 in WORDS:
  for w2 in WORDS:
    if w1 != w2:
      if w1 not in words_dist:
        words_dist[w1] = {"GN": 0, "BR": 0}
      words_dist[w1]["GN"] += w.distance(w1, w2)
      words_dist[w1]["BR"] += w_british.distance(f"{w1.lower()}_NOUN", f"{w2.lower()}_NOUN")

  words_dist[w1]["GN"] /= len(WORDS) -1
  words_dist[w1]["BR"] /= len(WORDS) -1

GN_unsimilar, BR_unsimilar = "", ""

for word, dist in words_dist.items():
  if dist["GN"] > words_dist.get(GN_unsimilar, {}).get("GN", 0):
    GN_unsimilar = word

  if dist["BR"] > words_dist.get(BR_unsimilar, {}).get("BR", 0):
    BR_unsimilar = word

print("GN:", GN_unsimilar)
print("BR:", BR_unsimilar)
words_dist

GN: television
BR: tiger


{'tiger': {'GN': 0.8869160364071528, 'BR': 0.8172915379206339},
 'plane': {'GN': 0.7763195087512335, 'BR': 0.7440997809171677},
 'train': {'GN': 0.7955030103524526, 'BR': 0.7092982033888499},
 'television': {'GN': 0.9142811397711436, 'BR': 0.791464830438296}}

### 3. Определите косинусное расстояние между векторами предложений: (Дисклеймер: из оригинальных пословиц удлаены слова, отсутствующие в модели GN.)
`bad workman always blames his tools` и `bird in hand is worth two in the bush`

Для нахождения вектора предложения требуется найти вектор каждого слова в предложении, а затем усреднить эти векторы покомпонентно (рекомендуем использовать numpy.mean() с правильным параметром axis).

Введите результат модели GN:

In [88]:
def count_sentance_dist(sentance):
  vector = []
  for word in sentance.split():
      vector.append(w[word].copy())
  return np.array(vector).mean(0)

def cosine_distance(a, b):
    cosine_sim = np.linalg.norm(a) * np.linalg.norm(b)
    cosine_sim = 0.0 if cosine_sim == 0.0 else np.dot(a, b) / cosine_sim
    return 1 - cosine_sim


SENTANCE_1 = "bad workman always blames his tools"
SENTANCE_2 = "bird in hand is worth two in the bush"


print("%.3f" % cosine_distance(count_sentance_dist(SENTANCE_1), count_sentance_dist(SENTANCE_2)))

0.621


### 4. Из набора данных word_sim извлеките подвыборку пар слов с индексами 10:110 (нумерация начинается с нуля, правая граница не включается).

Из уже выбранных пар используйте только те, для слов которых находятся векторы на британском корпусе, помеченные как существительные! Иначе удаляйте такую пару из подвыборки.

Вычислите корреляцию Спирмена между оценками схожести выбранных пар слов, полученных в результате работы моделей, и оценками аннотаторов в датасете word_sim.

In [94]:
gn_dist, br_dist, scores = [], [], []
sub_df = df[10:110]
count_deleted = 0
for row in sub_df.iterrows():

  w1, w2, s = row[1]["first"], row[1]["second"], row[1]["score"]
  try:
    #enter your code here
    word_1 = f"{w1.lower()}_NOUN"
    word_2 = f"{w2.lower()}_NOUN"
    br_dist.append(w_british.distance(word_1, word_2))
    gn_dist.append(w.distance(w1, w2))
    scores.append(s)

  except KeyError as e:
    count_deleted += 1
    # print(e, "Skipping this word.")

#enter your code here
print("GN spearmanr corr: %.3f" % abs(spearmanr(gn_dist, scores).statistic))
print("British spearmanr corr: %.3f" % abs(spearmanr(br_dist, scores).statistic))
print("Слов удалено: ", count_deleted)

GN spearmanr corr: 0.669
British spearmanr corr: 0.629
Слов удалено:  4
